In [6]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [7]:
import winsound
frequency = 2500  # set frequency to 2500 Hertz
duration = 1000   # set Duration to 1000 ms == 1 second
import numpy as np
import cv2

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

cap = cv2.VideoCapture(1)
# Check if the webcam is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Load the pre-trained models (new_model1 for eye prediction and new_model2 for mouth prediction)
new_model1 = tf.keras.models.load_model('SubmitEpochs10.h5')
new_model2 = tf.keras.models.load_model('Yawnepochs10.h5')

counter = 0   

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    # Initialize eye_state to a default value (representing open eyes)
    eye_state = 0.5

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Detect eyes within the face region
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

            # Preprocess the eye region and pass it to new_model1 for eye state prediction
            eye_img = roi_color[ey:ey+eh, ex:ex+ew]
            eye_img = cv2.resize(eye_img, (224, 224))
            eye_img = np.expand_dims(eye_img, axis=0)
            eye_img = eye_img / 255.0

            prediction_eye = new_model1.predict(eye_img)
            eye_state = prediction_eye[0][0]  # Extract the prediction value for eye state

        # Detect mouth within the face region
        mouths = mouth_cascade.detectMultiScale(roi_gray)
        for (mx, my, mw, mh) in mouths:
            cv2.rectangle(roi_color, (mx, my), (mx+mw, my+mh), (0, 0, 255), 2)

            # Preprocess the mouth region and pass it to new_model2 for mouth state prediction
            mouth_img = roi_color[my:my+mh, mx:mx+mw]
            mouth_img = cv2.resize(mouth_img, (224, 224))
            mouth_img = np.expand_dims(mouth_img, axis=0)
            mouth_img = mouth_img / 255.0

            prediction_mouth = new_model2.predict(mouth_img)
            mouth_state = prediction_mouth[0][0]  # Extract the prediction value for mouth state

            # Based on the eye and mouth states, trigger the appropriate alarm
            if eye_state < 0.5:  # Eye closed
                if mouth_state > 0.5:  # Yawn detected
                    # High volume alarm with red sleep alert
                    winsound.Beep(frequency, duration)
                    cv2.putText(frame, 'Sleep Alert !!', (x+int(w/10), y+int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    # Medium volume alarm with yellow sleep alert
                    winsound.Beep(frequency, duration)
                    cv2.putText(frame, 'Sleep Alert !!', (x+int(w/10), y+int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow('Drowsiness Detection Tutorial', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
